In [1]:
!pip install -q transformers peft accelerate bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sat Aug  2 13:04:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
from huggingface_hub import login
login()

In [4]:
from datasets import load_dataset

raw_ds = load_dataset("FinGPT/fingpt-fiqa_qa")

def format_gemma_instruction(example):
    user_prompt = f"{example['instruction'].strip()}\n\n{example['input'].strip()}"
    model_response = example['output'].strip()
    formatted = f"<start_of_turn>user\n{user_prompt}<end_of_turn>\n<start_of_turn>model\n{model_response}<end_of_turn>"
    return {"text": formatted}

formatted_ds = raw_ds["train"].map(format_gemma_instruction)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

(…)-00000-of-00001-ab79bf9300210e98.parquet:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17110 [00:00<?, ? examples/s]

Map:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it", token=True)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_ds = formatted_ds.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Map:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    load_in_4bit=True,
    device_map="auto",
    token=True
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.0368


In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gemma-finance-lora",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling since we're doing causal LM
)

# Use only a smaller subset of the dataset for quicker training (e.g., 2000 samples)
small_train_dataset = tokenized_ds.select(range(200))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()



/tmp/ipython-input-3660813980.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,5.023100
20,5.179000
30,4.848200
40,4.958300
50,4.271100
60,4.128300
70,4.283500
80,4.119500
90,4.070800
100,3.996200


TrainOutput(global_step=300, training_loss=3.9497398885091144, metrics={'train_runtime': 684.7515, 'train_samples_per_second': 0.876, 'train_steps_per_second': 0.438, 'total_flos': 3654708048691200.0, 'train_loss': 3.9497398885091144, 'epoch': 3.0})

In [10]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

instruction = "Utilize your financial knowledge, give your answer to the input:\n\nWhat are the benefits of portfolio diversification?"
prompt = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"

response = pipe(prompt, max_new_tokens=200)[0]['generated_text']

print(response)


Device set to use cuda:0


<start_of_turn>user
Utilize your financial knowledge, give your answer to the input:

What are the benefits of portfolio diversification?<end_of_turn>
<start_of_turn>model
A sharpe ratio is a measure of the risk and reward of an investment portfolio. A sharpe ratio greater than 1 indicates that the portfolio is more likely to have higher risk, but also more likely to have higher returns. A sharpe ratio less than 1 indicates that the portfolio is more likely to have lower risk, but also more likely to have lower returns. The sharpe ratio is a useful measure of an investment portfolio's risk and reward. It can help investors to decide how to allocate their investment dollars.

The benefit of portfolio diversification is that it reduces risk. When you spread your investment dollars across multiple investments, you spread the risk out. This means that if one investment falls in value, the other investments will still be worth something. This is a great way to protect your investment dollar

##Analyzing annual reports

In [11]:
from transformers import pipeline

# Set up your pipeline (assuming model and tokenizer are already loaded)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def analyze_financial_text(text_chunk):
    instruction = (
        "You are a financial analyst. Read the following financial text and summarize key insights, risks, and opportunities:\n\n"
        f"{text_chunk}"
    )
    prompt = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"
    return pipe(prompt, max_new_tokens=300)[0]['generated_text']


Device set to use cuda:0


In [12]:
sample_text = """
In fiscal year 2023, the company experienced a 12% increase in net revenue, primarily driven by the growth in cloud services and enterprise solutions.
Operating expenses rose by 7%, largely due to increased R&D investment. International markets contributed 45% of total revenue.
However, geopolitical tensions and rising interest rates posed challenges in key markets.
"""

output = analyze_financial_text(sample_text)
print(output)


<start_of_turn>user
You are a financial analyst. Read the following financial text and summarize key insights, risks, and opportunities:


In fiscal year 2023, the company experienced a 12% increase in net revenue, primarily driven by the growth in cloud services and enterprise solutions.
Operating expenses rose by 7%, largely due to increased R&D investment. International markets contributed 45% of total revenue.
However, geopolitical tensions and rising interest rates posed challenges in key markets.
<end_of_turn>
<start_of_turn>model
**Insights:**
- Company's growth in cloud services and enterprise solutions.
- International market contribution of 45%.
- Geopolitical tensions and rising interest rates posed challenges in key markets.
- Increased R&D investment.


**Risks:**
- Dependence on cloud services and enterprise solutions.
- Geopolitical tension and rising interest rates in key markets.
- Dependence on international markets.


**Opportunities:**
- Growing demand for cloud ser

##Evaluating a user’s current investment portfolio

In [13]:
def analyze_portfolio(text_portfolio):
    instruction = (
        "You are a financial advisor. Analyze the following investment portfolio and provide insights on:\n"
        "- Diversification\n"
        "- Sectoral exposure\n"
        "- Asset class risk\n"
        "- Growth potential\n\n"
        f"Portfolio:\n{text_portfolio}"
    )
    prompt = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"
    return pipe(prompt, max_new_tokens=300)[0]['generated_text']


In [14]:
user_portfolio = """
The user holds the following:
- 40% in large-cap US tech stocks (Apple, Microsoft, Nvidia)
- 20% in government bonds (10-year US Treasury)
- 15% in REITs (Real Estate Investment Trusts)
- 10% in emerging market equities
- 10% in gold
- 5% in a cryptocurrency ETF
"""

response = analyze_portfolio(user_portfolio)
print(response)


<start_of_turn>user
You are a financial advisor. Analyze the following investment portfolio and provide insights on:
- Diversification
- Sectoral exposure
- Asset class risk
- Growth potential

Portfolio:

The user holds the following:
- 40% in large-cap US tech stocks (Apple, Microsoft, Nvidia)
- 20% in government bonds (10-year US Treasury)
- 15% in REITs (Real Estate Investment Trusts)
- 10% in emerging market equities
- 10% in gold
- 5% in a cryptocurrency ETF
<end_of_turn>
<start_of_turn>model
## Insights on Investment Portfolio

**Diversification:**
The portfolio has a very high diversification among different asset classes and industries. This is a good approach to mitigate the risk of losing money in one specific sector or asset class. However, the portfolio is very concentrated in the tech sector, with a large allocation to Apple, Microsoft, and Nvidia. This could be a risk if these companies perform poorly.

**Sectoral Exposure:**
The portfolio has a significant allocation to

##Generating strategic suggestions

In [15]:
def generate_investment_suggestions(text_portfolio):
    instruction = (
        "You are a financial strategist. Based on the portfolio provided below, give:\n"
        "- Rebalancing suggestions\n"
        "- Stocks or sectors to add to a watchlist\n"
        "- Flag any underperforming or overexposed sectors\n"
        "- Suggestions should reflect current financial market trends (assume up-to-date knowledge).\n\n"
        f"Portfolio:\n{text_portfolio}"
    )
    prompt = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"
    return pipe(prompt, max_new_tokens=350)[0]['generated_text']

In [16]:
portfolio = """
40% Apple, Microsoft, Nvidia (US Tech)
20% US Treasury Bonds
10% Gold
10% India Small Cap Stocks
10% Real Estate REITs
10% Cash
"""

suggestions = generate_investment_suggestions(portfolio)
print(suggestions)

<start_of_turn>user
You are a financial strategist. Based on the portfolio provided below, give:
- Rebalancing suggestions
- Stocks or sectors to add to a watchlist
- Flag any underperforming or overexposed sectors
- Suggestions should reflect current financial market trends (assume up-to-date knowledge).

Portfolio:

40% Apple, Microsoft, Nvidia (US Tech)
20% US Treasury Bonds
10% Gold
10% India Small Cap Stocks
10% Real Estate REITs
10% Cash
<end_of_turn>
<start_of_turn>model
**Rebalancing Suggestions:**
- Consider adding some US stocks, like Tesla or Amazon.
- Consider adding some international stocks, like Alibaba or Tencent.
- Consider adding some smaller US REITs.

**Stocks to add to a watchlist:**
- TSLA
- AAPL
- MSFT
- GOOGL
- INOC
- INTC
- FLTR

**Flag any underperforming or overexposed sectors:**
- Consider adding some small US stocks to a watchlist.
- Consider adding some international stocks to a watchlist.

**Suggestions:**
- Consider adding some US stocks to your portfoli

In [17]:
model.save_pretrained("./gemma-finance-lora")
tokenizer.save_pretrained("./gemma-finance-lora")

('./gemma-finance-lora/tokenizer_config.json',
 './gemma-finance-lora/special_tokens_map.json',
 './gemma-finance-lora/chat_template.jinja',
 './gemma-finance-lora/tokenizer.model',
 './gemma-finance-lora/added_tokens.json',
 './gemma-finance-lora/tokenizer.json')

In [18]:
!pip install streamlit sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [31]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

st.set_page_config(page_title="Finance Assistant", layout="wide")

# Load model and tokenizer
@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("./gemma-finance-lora")
    model = AutoModelForCausalLM.from_pretrained(
        "./gemma-finance-lora",
        torch_dtype=torch.float16,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_model()

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(output[0])  # Don’t skip special tokens

def extract_model_answer(response):
    try:
        if "<start_of_turn>model" in response and "<end_of_turn>" in response:
            answer = response.split("<start_of_turn>model")[1].split("<end_of_turn>")[0].strip()
            return answer
        else:
            return None
    except Exception as e:
        return None

# Streamlit UI
st.title("📊 Financial LLM Assistant")

mode = st.sidebar.radio("Select Task", ["Q&A", "Analyze Annual Report", "Evaluate Portfolio", "Get Suggestions"])

def handle_response(prompt_text):
    st.info("Generating response...")
    response = generate_response(prompt_text)
    answer = extract_model_answer(response)
    if answer:
        st.markdown("**Answer:**")
        st.write(answer)
    else:
        st.warning("⚠️ Model response not in expected format. Displaying raw output:")
        st.code(response)

if mode == "Q&A":
    user_query = st.text_area("💬 Enter your financial question:")
    if st.button("Submit"):
        if user_query.strip():
            prompt = f"<start_of_turn>user\n{user_query}<end_of_turn>\n<start_of_turn>model\n"
            handle_response(prompt)

elif mode == "Analyze Annual Report":
    report = st.text_area("📄 Paste a section of an annual report:")
    if st.button("Analyze"):
        if report.strip():
            prompt = f"<start_of_turn>user\nSummarize this section from an annual report:\n{report}<end_of_turn>\n<start_of_turn>model\n"
            handle_response(prompt)

elif mode == "Evaluate Portfolio":
    portfolio = st.text_area("📈 Paste your investment portfolio (e.g., tickers, sectors, weights):")
    if st.button("Evaluate"):
        if portfolio.strip():
            prompt = f"<start_of_turn>user\nEvaluate this portfolio:\n{portfolio}<end_of_turn>\n<start_of_turn>model\n"
            handle_response(prompt)

elif mode == "Get Suggestions":
    portfolio = st.text_area("🧠 Paste your portfolio to get improvement suggestions:")
    if st.button("Suggest"):
        if portfolio.strip():
            prompt = f"<start_of_turn>user\nGive strategic suggestions for this portfolio including rebalancing, underperformers, and watchlist ideas:\n{portfolio}<end_of_turn>\n<start_of_turn>model\n"
            handle_response(prompt)


Overwriting app.py


In [32]:
!pip install pyngrok
!ngrok config add-authtoken 30jXw8LZ4E9XoWOx3YI7vjD3vne_32ik6279ZmxsQxjBpsJEs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Open your app at:", public_url)

!streamlit run app.py &

Open your app at: NgrokTunnel: "https://3d7344867593.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.231.226.208:8501

2025-08-02 14:09:17.074171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754143757.196892   17713 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754143757.242275   17713 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:28<00:00, 14.29s/it]
  Stopping...
